<a href="https://colab.research.google.com/github/kavyajeetbora/recipe_recommender/blob/master/notebooks/food_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer
import torch
from tqdm.notebook import tqdm
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

## Read the data
nutrition - calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat (PDV) ,carbohydrates (PDV)

In [4]:
%%time
df = pd.read_csv('C:\\Users\\Mina Mahros\\Desktop\\Grauation Project\\recipe-personalization\\recipe_recommender\\csv files\\RAW_recipes.csv')
df = df.reset_index(drop=True)
df.head()

CPU times: total: 3.7 s
Wall time: 3.76 s


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


## Transform the data

In [6]:
## Split the nutrition column to separate columns
df[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df.nutrition.str.split(",",expand=True)

## Replace [] brackets after split
df['calories'] =  df['calories'].apply(lambda x: x.replace('[',''))
df['carbohydrates (PDV)'] =  df['carbohydrates (PDV)'].apply(lambda x: x.replace(']',''))

## Convert the datatypes from str to float
df[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']].astype('float')

## Convert steps to list format
df['steps'] = df['steps'].str.strip("[]").str.replace('"',"").str.split("',").apply(lambda x: [y.replace("'","").strip() for y in x])

## Drop unnecessary columns
df.drop(['id', 'contributor_id','submitted','tags','nutrition'], axis=1,inplace = True)

In [7]:
df.dtypes

name                    object
minutes                  int64
n_steps                  int64
steps                   object
description             object
ingredients             object
n_ingredients            int64
calories               float64
total fat (PDV)        float64
sugar (PDV)            float64
sodium (PDV)           float64
protein (PDV)          float64
saturated fat (PDV)    float64
carbohydrates (PDV)    float64
dtype: object

In [8]:
df.head()

,name,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,arriba baked winter squash mexican style,55,11,"[make a choice and proceed with recipe, depend...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,30,9,"[preheat oven to 425 degrees f, press dough in...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,130,6,"[brown ground beef in large pot, add chopped o...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,45,11,[place potatoes in a large pot of lightly salt...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,amish tomato ketchup for canning,190,5,"[mix all ingredients& boil for 2 1 / 2 hours ,...",my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0


## Export the data to a parquet file

In [9]:
df.to_parquet('C:\\Users\\Mina Mahros\\Downloads\\parquet\\RAW_recipes.parquet')

## Cleaning the ingredients column

In [10]:
%%time
## First convert the string representation of list to list
df['ingredients'] = df['ingredients'].str.strip("[]").str.replace("'","") #.str.split(",")

CPU times: total: 312 ms
Wall time: 349 ms


In [11]:
df['ingredients'].iloc[0]

'winter squash, mexican seasoning, mixed spice, honey, butter, olive oil, salt'

Explode the series containing list to a single list:

In [12]:
ingredients = df['ingredients'].explode().str.lower().str.strip()
print(f"Around {ingredients.shape[0]} ingredients are there in the dataset")

Around 231637 ingredients are there in the dataset


Now find the unique ingredients:

In [13]:
unique_ingredients = ingredients.unique()
print(f"There around {unique_ingredients.shape[0]} are unique out of {ingredients.shape[0]}")

There around 230475 are unique out of 231637


There might be some same words that mean the same but with different name.

Try with some keywords to see if it exists like search for 'egg' or 'milk'

In [14]:
keyword = 'bread'
unique_word = ingredients[ingredients.str.contains(keyword.lower())].unique()
if len(unique_word)> 0:
    sample = ", ".join(np.random.choice(unique_word,10))
    print(f"There are around {len(unique_word)} unique ingredients containing the work 'f{keyword}'.\n\n Example:\n\n {sample}....")
else:
    print(f"No ingredients for the keyword: {keyword}")

There are around 18673 unique ingredients containing the work 'fbread'.

 Example:

 water, light brown sugar, ground ginger, salt, bread flour, whole wheat flour, active dry yeast, cornmeal, crusty bread, eggs, milk, salt and pepper, sweet paprika, vegetarian sausage, cheddar cheese, fresh mushrooms, all-purpose flour, seasoned bread crumbs, egg, milk, parmesan cheese, pita breads, cucumber, salt, tomatoes, green onions, green bell pepper, flat leaf parsley, mint leaf, fresh cilantro leaves, garlic cloves, fresh lemon juice, olive oil, fresh ground black pepper, water, onion, butter, bread flour, poppy seeds, sugar, dry milk, salt, bread machine yeast, olive oil, yellow onion, cremini mushroom, garlic cloves, white bread, whole milk, italian sausage, ground beef, eggs, lemon zest, fresh basil, fresh thyme, garam masala, worcestershire sauce, kosher salt, fresh ground black pepper, bacon, cornbread, olive oil, fresh thyme leaves, bulk pork sausage, unsalted butter, onions, celery, drie

## Sentence encoder model


Selecting the encoder model: https://www.sbert.net/docs/pretrained_models.html

In [15]:
%%time
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

CPU times: total: 750 ms
Wall time: 2.41 s


In [16]:
# Set the device
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

## Encoding the ingredients

In [17]:
%%time

## encode all the ingredients:
embedding = model.encode(df['ingredients'].values)

CPU times: total: 1h 28min 26s
Wall time: 22min 26s


In [18]:
df_final = df.copy()
df_final['embedding'] = list(zip(embedding))
df_final['embedding'] = df_final['embedding'].apply(lambda x: x[0])
df_final.head()

,name,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),embedding
0,arriba baked winter squash mexican style,55,11,"[make a choice and proceed with recipe, depend...",autumn is my favorite time of year to cook! th...,"winter squash, mexican seasoning, mixed spice,...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0,"[-0.008845084, 0.00986693, 0.028063837, 0.1012..."
1,a bit different breakfast pizza,30,9,"[preheat oven to 425 degrees f, press dough in...",this recipe calls for the crust to be prebaked...,"prepared pizza crust, sausage patty, eggs, mil...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0,"[-0.054575548, 0.027983472, 0.06526372, 0.0321..."
2,all in the kitchen chili,130,6,"[brown ground beef in large pot, add chopped o...",this modified version of 'mom's' chili was a h...,"ground beef, yellow onions, diced tomatoes, to...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0,"[-0.104012504, -0.024387976, 0.06653507, 0.034..."
3,alouette potatoes,45,11,[place potatoes in a large pot of lightly salt...,"this is a super easy, great tasting, make ahea...","spreadable cheese with garlic and herbs, new p...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0,"[-0.04868247, -0.046541482, -0.013404083, 0.01..."
4,amish tomato ketchup for canning,190,5,"[mix all ingredients& boil for 2 1 / 2 hours ,...",my dh's amish mother raised him on this recipe...,"tomato juice, apple cider vinegar, sugar, salt...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0,"[-0.06817172, 0.030255308, 0.026339037, 0.0327..."


In [19]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   name                 231636 non-null  object 
 1   minutes              231637 non-null  int64  
 2   n_steps              231637 non-null  int64  
 3   steps                231637 non-null  object 
 4   description          226658 non-null  object 
 5   ingredients          231637 non-null  object 
 6   n_ingredients        231637 non-null  int64  
 7   calories             231637 non-null  float64
 8   total fat (PDV)      231637 non-null  float64
 9   sugar (PDV)          231637 non-null  float64
 10  sodium (PDV)         231637 non-null  float64
 11  protein (PDV)        231637 non-null  float64
 12  saturated fat (PDV)  231637 non-null  float64
 13  carbohydrates (PDV)  231637 non-null  float64
 14  embedding            231637 non-null  object 
dtypes: float64(7), in

## Export the embedded recipes to parquet files

In [20]:
partitions = np.linspace(0, df_final.shape[0], 30, dtype=int)
for ix in range(len(partitions)-1):
    i,j = partitions[ix], partitions[ix+1]
    dfx = df_final.iloc[i:j,:]
    dfx.to_parquet(rf'C:\Users\Mina Mahros\Downloads\parquet\Exports\recipes_{ix}.parquet')

## Finally calculate the cosine similarity

In [21]:
def cosine_similarity(vec1,vec2):
    '''
    Returns the cosine similarity between two vectors of n dimension
    '''
    denom = np.sqrt(np.sum(np.square(vec1))) * np.sqrt(np.sum(np.square(vec2)))
    return np.round(np.dot(vec1,vec2) / denom * 100, 2)

In [22]:
index = 3
data = df_final.iloc[index]
recipe, vector = data['name'], data['embedding']

print("Name of the dish:", recipe)

Name of the dish: alouette  potatoes


In [23]:
%%time

df_result = df_final.copy()
df_result['similarity'] = df_result['embedding'].apply(lambda x : cosine_similarity(vector, x))
df_result.sort_values(by="similarity", ascending=False).iloc[1:4]

CPU times: total: 6.64 s
Wall time: 6.68 s


,name,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),embedding,similarity
62541,creamy cheesy scrambled eggs,10,7,"[over medium to medium low heat , melt butter ...",these are not your ordinary scrambled eggs. i ...,"eggs, cheddar cheese, butter, light spreadable...",5,259.2,29.0,3.0,12.0,38.0,42.0,0.0,"[-0.0654695, 0.0023903176, 0.02905353, 0.07523...",85.22
52319,chopped caprese salad,20,10,"[1, combine mozzarella , oil , shallot , garli...",traditional mozzarella-tomato-basil taken up a...,"fresh mozzarella cheese, extra virgin olive oi...",10,240.9,25.0,23.0,12.0,24.0,30.0,4.0,"[-0.076132834, -0.00777417, 0.049261022, 0.040...",84.25
213211,three onion casserole,80,15,"[preheat oven to 350f, butter an 8 cup baking ...","from the silver palate good times cookbook, th...","butter, yellow onions, red onions, leeks, salt...",10,254.7,22.0,27.0,14.0,20.0,44.0,6.0,"[-0.07928342, 0.025039831, -0.020423688, 0.065...",84.03
